In [1]:
# from transformers import AutoTokenizer, EsmForSequenceClassification
import torch

In [2]:
# Can be used
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t6_8M_UR50D")

Using cache found in /home/nikola_dev/.cache/torch/hub/facebookresearch_esm_main
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /home/nikola_dev/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /home/nikola_dev/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


In [3]:
import esm

In [81]:
model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()

In [82]:
# model has a base class nn.Module
print(esm.model.esm2.ESM2.__bases__)
model

(<class 'torch.nn.modules.module.Module'>,)


ESM2(
  (embed_tokens): Embedding(33, 640, padding_idx=1)
  (layers): ModuleList(
    (0-29): 30 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=640, out_features=640, bias=True)
        (v_proj): Linear(in_features=640, out_features=640, bias=True)
        (q_proj): Linear(in_features=640, out_features=640, bias=True)
        (out_proj): Linear(in_features=640, out_features=640, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=640, out_features=2560, bias=True)
      (fc2): Linear(in_features=2560, out_features=640, bias=True)
      (final_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=600, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((640,), eps=1e-05, elementw

In [4]:
from torch.nn import Identity
model.contact_head = Identity()
model.emb_layer_norm_after = Identity()
model.lm_head = Identity()
model

ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): Identity()
  (emb_layer_norm_after): Identity()
  (lm_head): Identity()
)

In [5]:
model.eval()
batch_converter = alphabet.get_batch_converter()
data = [
    ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
    ("protein2", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG")
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

In [20]:
with torch.no_grad():
    res = model(batch_tokens, repr_layers=[6])

In [21]:
res

{'logits': tensor([[[ 2.5057,  6.8665,  1.0299,  ...,  8.5440, -1.4410, -7.0620],
          [ 4.4105,  2.9949, -1.7162,  ...,  4.4722, -0.3008, -2.9338],
          [-0.7679, -2.4428, -1.0123,  ..., -1.6110,  2.6296,  0.2919],
          ...,
          [-1.7249, -1.8000,  4.2837,  ...,  1.0689, -0.8234, -5.0085],
          [-0.1915, -0.7245,  2.4475,  ..., -0.7194,  0.2490, -4.1189],
          [-1.1293,  1.0655,  1.3112,  ...,  1.8803, -6.6340, -4.0444]],
 
         [[ 2.5057,  6.8665,  1.0299,  ...,  8.5440, -1.4410, -7.0620],
          [ 4.4105,  2.9949, -1.7162,  ...,  4.4722, -0.3008, -2.9338],
          [-0.7679, -2.4428, -1.0123,  ..., -1.6110,  2.6296,  0.2919],
          ...,
          [-1.7249, -1.8000,  4.2837,  ...,  1.0689, -0.8234, -5.0085],
          [-0.1915, -0.7245,  2.4475,  ..., -0.7194,  0.2490, -4.1189],
          [-1.1293,  1.0655,  1.3112,  ...,  1.8803, -6.6340, -4.0444]]]),
 'representations': {6: tensor([[[ 2.5057,  6.8665,  1.0299,  ...,  8.5440, -1.4410, -7.06